In [30]:
import numpy as np
from datetime import datetime
import plotly
import plotly.graph_objs as go
import chart_studio.plotly as py 
import random
import math

# f(LAT,LON) = r1 * math.e ^ ( - r2*(LAT+r3)^2 - r4*(LON+r5)^2 )
Deltas = list()
for i in range( 0, 200 ):
    r1 = 0.1 * ( random.random() * 1 - 0.5 )  # gives spike height
    r2 = random.random() * 0.01 # spike width at Latitudes
    r3 = random.random() * 180 - 90 # spike position at Latitudes
    r4 = random.random() * 0.01 # spike width at Longitudes
    r5 = random.random() * 360 - 180 # spike position at Longitudes
    Deltas.append( [r1,r2,r3,r4,r5] )
     

Lons = list( range( -180, 180, 2 ) )
Lats = list( range(  -90,  90, 2 ) )
Zaxis = np.zeros( (len(Lons), len(Lats)) ).tolist()
Data  = np.zeros( (len(Lons), len(Lats)) ).tolist()
for i in range( 0, len(Lons) ):
    for j in range( 0, len(Lats) ):
        LON = Lons[i]
        LAT = Lats[j]
        for D in Deltas:
            r1 = D[0]
            r2 = D[1]
            r3 = D[2]
            r4 = D[3]
            r5 = D[4]
            #if i==0 and j==0: print( - r2*(LAT+r3)**2 - r4*(LON+r5)**2 ,   math.e ** ( - r2*(LAT+r3)**2 - r4*(LON+r5)**2  ) )
            Data[i][j] +=  r1 * math.e ** ( - r2*(LAT+r3)**2 - r4*(LON+r5)**2  )
            Data[i][j] +=  r1 * math.e ** ( - r2*(LAT+r3)**2 - r4*(LON-360+r5)**2  )
            Data[i][j] +=  r1 * math.e ** ( - r2*(LAT+r3)**2 - r4*(LON+360+r5)**2  )
            #if i==0 and j==0: print(r1 * math.e ** ( - r2*(LAT+r3)**2 - r4*(LON+r5)**2  ), r1 * math.e ** ( - r2*(LAT+r3)**2 - r4*(LON-362+r5)**2  ))
        
#for i in range( int(len(Lons)/3), len(Lons) ):
#    for j in range( 0, len(Lats) ):
#        Data[i][j] += 60
        
# define the layout of the plot
theLayout = dict( title="Sub Grid Variability", width=1000, height=800,
        scene = dict(
            xaxis = dict(title="Altitude",dtick=10), yaxis = dict(title="Longitude",dtick=20), zaxis = dict( title="" ),
        )
)

surface=go.Surface(x=Lats, y=Lons, z=Data,   colorscale = "jet", surfacecolor = Data,
        colorbar=dict(thickness=20, len=0.75, ticklen=4, title="Value"),
)
Plotables = list()
Plotables.append( surface )
fig = dict( data=Plotables, layout=theLayout )
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)

surface=go.Surface(x=Lats, y=Lons, z=Zaxis,   colorscale = "jet", surfacecolor = Data,
        colorbar=dict(thickness=20, len=0.75, ticklen=4, title="Value"),
)
Plotables = list()
Plotables.append( surface )
fig = dict( data=Plotables, layout=theLayout )
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)


    

[-180, -178, -176, -174, -172, -170, -168, -166, -164, -162, -160, -158, -156, -154, -152, -150, -148, -146, -144, -142, -140, -138, -136, -134, -132, -130, -128, -126, -124, -122, -120, -118, -116, -114, -112, -110, -108, -106, -104, -102, -100, -98, -96, -94, -92, -90, -88, -86, -84, -82, -80, -78, -76, -74, -72, -70, -68, -66, -64, -62, -60, -58, -56, -54, -52, -50, -48, -46, -44, -42, -40, -38, -36, -34, -32, -30, -28, -26, -24, -22, -20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178]
